In [1]:
from pymilvus_orm import connections, FieldSchema, CollectionSchema, DataType, Collection
import h5py
import pymilvus_orm


In [2]:
files = ["./data/pdbaa-1.h5","./data/pdbaa-2.h5"]
f = h5py.File(files[0], 'r')

In [3]:
query_file = "./data/2.60.40.10.h5"
query_vector = list(h5py.File(query_file, 'r')["3kdmA02"])

In [4]:
connections.connect(host='localhost', port='19530')

In [6]:
collection_name = "pdb"
Collection(name=collection_name).drop()

In [6]:
dim = 6165

fields = [
    FieldSchema(name="enumerate_id", dtype=DataType.INT64, is_primary=True),
    #FieldSchema(name="sequence_id", dtype=DataType.STRING),
    FieldSchema(name="sequence_vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
schema = CollectionSchema(fields=fields, description="omics vectors")

In [7]:
collection = Collection(name=collection_name, schema=schema)

In [8]:
batch = [[],[]]
for j, key in enumerate(f.keys()):
    batch[0].append(j)
    batch[1].append([float(i) for i in f[key]])
    if len(batch[0]) >= 1000:
        collection.insert(batch)
        batch = [[],[]]
        break

In [9]:
collection.create_index(field_name="sequence_vector",
                        index_params={'index_type': 'IVF_FLAT',
                                      'metric_type': 'IP',
                                      'params': {
                                        'nlist': 100      # int. 1~65536
                                      }})


Status(code=0, message='')

In [10]:
connections.get_connection("default").list_collections()

['pdb']

In [11]:
connections.get_connection("default").flush([collection_name])
collection.num_entities

1000

In [9]:
collection.load()

1000

In [14]:
topK = 50
search_params = {"metric_type": "IP", "params": {"nprobe": 1}}
# define output_fields of search result
res = collection.search(
    [query_vector], "sequence_vector", search_params, topK,
    "", output_fields=["enumerate_id"]
)

In [15]:
for result in res:
    for id, dis in zip(result.ids, result.distances):
        print(f"id = {id}, distance = {dis}")

id = 624, distance = 52.20811462402344
id = 282, distance = 52.20811462402344
id = 626, distance = 52.20811462402344
id = 284, distance = 52.20811462402344
id = 286, distance = 52.20811462402344
id = 783, distance = 51.29606246948242
id = 800, distance = 51.29606246948242
id = 781, distance = 51.29606246948242
id = 464, distance = 47.139957427978516
id = 467, distance = 47.139957427978516
id = 941, distance = 47.114200592041016
id = 939, distance = 47.114200592041016
id = 936, distance = 47.0709342956543
id = 471, distance = 47.063777923583984
id = 468, distance = 47.063777923583984
id = 233, distance = 46.66118621826172
id = 955, distance = 46.42018127441406
id = 958, distance = 46.42018127441406
id = 608, distance = 46.286231994628906
id = 606, distance = 46.286231994628906
id = 101, distance = 46.086082458496094
id = 982, distance = 45.73291778564453
id = 713, distance = 45.728111267089844
id = 714, distance = 45.728111267089844
id = 54, distance = 45.7249755859375
id = 53, distance

In [70]:
connections.disconnect("default")

In [40]:
collection.drop()